# Ensemble Retriever | アンサンブル・レトリーバー`EnsembleRetriever`は、複数のリトリーバーのリストを入力として受け取り、それらの`get_relevant_documents()`メソッドの結果を集約して、[Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)アルゴリズムに基づいて結果を再ランク付けします。> The `EnsembleRetriever` takes a list of retrievers as input and ensemble the results of their `get_relevant_documents()` methods and rerank the results based on the [Reciprocal Rank Fusion](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf) algorithm.異なるアルゴリズムの強みを活用することで、`EnsembleRetriever`はどの単一のアルゴリズムよりも優れたパフォーマンスを発揮することができます。> By leveraging the strengths of different algorithms, the `EnsembleRetriever` can achieve better performance than any single algorithm.最も一般的なパターンは、スパースリトリバー（BM25のような）とデンスリトリバー（埋め込み類似性のような）を組み合わせることです。これは、それぞれの強みが補完し合うためです。これは「ハイブリッド検索」とも呼ばれています。スパースリトリバーはキーワードに基づいて関連する文書を見つけるのに優れており、デンスリトリバーは意味的な類似性に基づいて関連する文書を見つけるのに優れています。> The most common pattern is to combine a sparse retriever (like BM25) with a dense retriever (like embedding similarity), because their strengths are complementary. It is also known as "hybrid search". The sparse retriever is good at finding relevant documents based on keywords, while the dense retriever is good at finding relevant documents based on semantic similarity.

In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS

In [14]:
doc_list = [
    "I like apples",
    "I like oranges",
    "Apples and oranges are fruits",
]

# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 2

embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(doc_list, embedding)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 2})

# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.5, 0.5]
)

In [16]:
docs = ensemble_retriever.get_relevant_documents("apples")
docs

[Document(page_content='I like apples', metadata={}),
 Document(page_content='Apples and oranges are fruits', metadata={})]